# 05. WordCount 프로그램 자세히 보기

- 텍스트 데이터만을 처리하면, Map과 Reduce 부분만 구현하면 됨.
- 이미지, 동영상 등을 처리하려면, 아래 그림에 나와 있는 부분을 모두 구현이 필요함. 

![](hadoop05_01.jpg)

## 05-1 Map 클래스 다시 살펴보기

#### Mapper 클래스의 메소드들
- public void run(Mapper.Context context)
   - Mapper에서는 기본적으로 하나의 block(64M)을 처리
   - 텍스트의 경우 한줄씩 읽어서 map함수로 전달하고, 개발자는 한줄에 대한 처리만 구현함.
```
public void run(Context context) throws IOException, InterruptedException {
    setup(context);
    while (context.nextKeyValue()) {
       map(context.getCurrentKey(), context.getCurrentValue(), context);
    }
    cleanup(context);
}
```

- public void setup(Mapper.Context context)
    - map함수가 호출되기 전에 한번만 호출되는 메소드
    - map함수에서 공통적으로 사용되는 리소스를 할당하거나, 선행작업을 처리함.
    - 예) DB연결이나 분산캐시 파일을 오픈 등

- public void clean(Mapper.Context context)
    - setup함수와 반대 역할, map()함수가 완료된 후에 한번만 호출
    - 리소스 해제와 같은 작업을 처리함
    - 예) DB연결 닫기,  분산케시 파일 닫기 등
    
#### map() 함수의 입력값들
- 텍스트의 하나의 라인이 하나의 레코드
- 해당 라인의 파일오프셋( 파일 처음부터의 위치)가 key가 되고, LongWritable이 타임.
- 해당 라인의 value가 되고, Text 타임.

#### map() 함수의 출력값들
- Key와 Value 값이 출력됨.
- Key와 Value 의 타입이 정수인지, 문자열로 출력될지는 개발자 마음이기 때문에 프레임워크에 알려주어야 함.
- Key와 Value 의 타입은 Job 클래스의 setMapOutKeyClass(), setMapOutValueClass()함수를 통해서 알려줌.

## 05-2 MapReduce 프로그램에서 사용되는 변수 타임들

![](hadoop05_02.jpg)

- 하둡에서 사용되는 변수 타임들은 2가지 기능이 추가되어야 함.
    - 1) 네트워크로 연결된 서버들간에 송수신을 기능. 직렬화/역직렬화 => Writable 인터페이스
    - 2) key사용되는 변수들은 정렬 또는 같은 key로 그룹핑이 되기 위한 비교 기능 => WritableComprable 인터페이스
- hadoop의 기본 변수들은 이미 구현되어 있으면, 추가적인 변수 타임을 필요하면 위의 2개의 인터페이스를 구현 필요.

## 05-3 입력포맷(InputFormat)의 역할

- 입력으로 사용되는 파일의 포맷에 따라 데이터를 읽어오는 역할을 함.
- hadoop에서는 텍스트 파일, DB, Xml 등은 기본적으로 읽어올 수 있도록 구현되어 있고,  이미지나 동영상 같은 경우는 직접 구현이 필요함.

#### 제공되는 입력 포맷 클래스들
- TextInputFormat
    - 텍스트 파일 대상, *.gz의 압축파일도 처리함.
    - 라인 하나( \n, \r)가 하나의 입력 레코드로 처리
- KeyValueTextInputFormat
    - TextInputFormat와 비슷하지만, key와 value을 TAB과 같은 구분자를 이용할 수 있음
- SequenceFileInputFormat
    - 하둡의 고유 파일 포맷인 SequenceFile을 처리.
    - SequenceFile은 파일 내용에 key/Value 기반의 압축 지원 파일 포맷
    - 작은 파일들을 하나의 큰 파일로 만들때 사용함.
- DBInputFormat, XmlInputFormat 등도 있음. 

#### 맵 태스크 수의 결정 방식
- 입력 파일의 수
    - 입력파일의 위치를 파일 또는 디렉토리로 지정이 가능함.
    - 입력 파일의 수보다 작아질 수 없음.
    - 작은 파일이 많으면 비효율적임.
- 입력 파일의 크기
    - HDFS에서 파일은 block(64MB)로 나누어지고, 하나의 블록당 맵 테스크가 하나 할당.
    - 압축된 파일을  block이 여러개라도 하나의 맵 테스크로 할당됨.

## 05-4 컴바이너(Combiner)란? 

- 아래 2개의 그림을 비교하면 쉽게 이해됨.
- map()에서 나온 결과를 로컬에서 Reduce작업를 해서, reduce()로 넘겨주는 통신량을 줄여줌.
- 하둡의 MapReduce의 작업들은 대부분 네크워크에서 병목이기 때문에 성능 향상에 중요한 역할을 함.

![](hadoop05_03.jpg)
![](hadoop05_04.jpg)

- 수학적으로 표현해서, 교환법칙과 결합법칙이 만족할때만 combiner을 사용 가능함.
- value가 최소값이나 최대값 같은 경우는 가능하고 평균일때는 적용 불가능(  평균의 경우 약간 추가 코드를 적용하면 적용 가능함 )
- job.setCombinerClass( Reducer.class ) 함수로 등록함.

## 05-5 셔플링과 소팅 

- map()함수의 출력값을 reduce()함수의 입력값으로 넘겨주는 과정

#### 파티셔너(Partitioner) - 레코드를 어느 리듀스 테스트로 보낼까?
- map()함수의 결과로 key/Value로 나오고, Key값이 동일한 것을 동일한 리듀서로 보내는 역할을 함.
- 디폴트 클래스인 HashPartitioner는  key값을 해싱값으로 변환해서  리듀스 태스트의 수로 나누어줌.
```
public int getPartition(K2 key, V2 value, int numReduceTasks) {
   return (key.hashCode() & Integer.MAX_VALUE) % numReduceTasks;
}
```

#### 셔플링
![](hadoop05_05.jpg)


#### Sorting
- combiner를 설명한 그림을 참고
- key로 소팅이 되므로, key값을 비교가 가능한 타임이여야 해서 WritableComprable 인터페이스를 구현함.

## 05-6 Reduce 클래스 자세히 보기

- map 클래스와 비슷하게 run(), setup(), cleanup() 메소드를 갖음.
```
public class  Reducer<KEYIN,VALUEIN,KEYOUT,VALUEOUT> {
public void run(Context context) throws IOException, InterruptedException {
    setup(context);
    while (context.nextKey()) {
      reduce(context.getCurrentKey(), context.getValues(), context);
    }
    cleanup(context);
  }
 }
```

#### reduce() 함수의  입력값
- 맵의 출력값들을 합쳐지고, 맵의 출력의 Key값과 맵의 출력의 Value값의 리스트를 입력으로 받음.
- combiner를 설명한 그림을 참고

#### Reducer클래스의  출력
- 기본적으로 텍스트 파일로 출력한다면, TextOutputFormat 클래스를 이용함.
- TextOutputFormat 클래스
    - 한 라인이 한 레코드에 해당, 출력은 텍스트 포맷
    -  reduce() 함수의 출력값의 key/value을  TAB문자로 구분해서 한 라인으로 저장함.
- 여러가지 OutputFormat 클래스가 있음.

## 05-7  출력포맷
- TextOutputFormat
- SequenceFileOutpuForamt
- MapFileOutputFormat
- MultipleOutputs

## 05-8 카운터

- Map과 Reduce에서 뭔가를 카운트할때 사용함.
- 특정 문자가 나온 횟수나 에러 횟수 등..

## 05-9 MRunit과 메이븐

```
   @Test
   public void testMapper() {
      mapDriver.withInput(new LongWritable(1), new Text("cat cat dog"));
      mapDriver.withOutput(new Text("cat"), new LongWritable(1));
      mapDriver.withOutput(new Text("cat"), new LongWritable(1));
      mapDriver.withOutput(new Text("dog"), new LongWritable(1));
      mapDriver.runTest();
   }

   @Test
   public void testReducer() {
      List<LongWritable> values = new ArrayList<LongWritable>();
      values.add(new LongWritable(1));
      values.add(new LongWritable(1));
      reduceDriver.withInput(new Text("cat"), values);
      reduceDriver.withOutput(new Text("cat"), new LongWritable(2));
      reduceDriver.runTest();
   }

   @Test
   public void testMapReduce() {
      mapReduceDriver.withInput(new LongWritable(1), new Text("cat cat dog"));
      mapReduceDriver.addOutput(new Text("cat"), new LongWritable(2));
      mapReduceDriver.addOutput(new Text("dog"), new LongWritable(1));
      mapReduceDriver.runTest();
   }
```




## 05-10 잡 트래커 웹 인터페이스